<a href="https://colab.research.google.com/github/vyphamhung10/khoa_luan/blob/master/price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
# region Import
# Data download
# Import basic
import csv
import math
import os
import warnings
# Init google drive
# from google.colab import drive
from datetime import datetime
from timeit import default_timer as timer

import numpy as np
import pandas as pd
# Plottool
import plotly.graph_objs as go
# IPython
from IPython.display import display
# Hyperopt bayesian optimization
from hyperopt import hp, Trials, tpe, fmin, STATUS_OK, partial
# Keras
import tensorflow as tf
import tensorflow 
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.activations import softmax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint  
from tensorflow.keras.initializers import random_normal, Ones 
from tensorflow.keras.layers import LSTM, Dropout, Input, Dense
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
# SKLearn
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# endregion

In [2]:
try:
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False

current_timestamp = datetime.now().strftime('%d%m%Y_%H%M%S')
if not IN_COLAB:
    
    # region File mount and config
    # drive.mount('/content/gdrive', force_remount=True)
    root_dir = ""
    
    time_dir = os.path.join(root_dir, "result")
    time_dir = os.path.join(time_dir, current_timestamp)
    
    data_dir = root_dir + 'data'
    model_dir = os.path.join(time_dir, 'model')
    plot_dir = os.path.join(time_dir, 'plot')
    result_dir = os.path.join(time_dir, 'result')
    # Create folder if not exists
    
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
        
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
else:
    # region File mount and config
    drive.mount('/content/gdrive', force_remount=True)
    root_dir = "/content/gdrive/My Drive/stock"
    
    time_dir = os.path.join(root_dir, "result")
    
    data_dir = os.path.join(root_dir, "data")
    model_dir = os.path.join(time_dir, 'model')
    plot_dir = os.path.join(time_dir, 'plot')
    result_dir = os.path.join(time_dir, 'result')
    
pd.options.display.max_columns = 12
pd.options.display.max_rows = 24

# disable warnings in Anaconda
warnings.filterwarnings('ignore')

# endregion

Mounted at /content/gdrive


In [3]:
!pip install yfinance
import yfinance as yf
# region Data Loading
stock_name = '000002.SS'  # SSE Composite Index

df_org = yf.download(stock_name, start="2006-01-01", end="2016-10-19", interval="1d")
#df_org = pd.read_csv(f'{data_dir}/{stock_name}.csv', parse_dates=['Date'])
df_org = df_org.sort_values('Date')
# df_org.to_csv(f'{base_dir}/{stock_name}.csv')
df_org.reset_index(inplace=True)
df_org = df_org[['Date', 'Close', 'Open', 'High', 'Low', 'Adj Close', 'Volume']]

# endregion

[*********************100%***********************]  1 of 1 completed


In [0]:
df_change = df_org['Close'].copy()
df_change = df_change.pct_change(periods=1, fill_method='ffill')
df_change = df_change.fillna(0)

df_org['Change'] = df_change

In [5]:
# region Data ploting
def plot_ohlc(df):
    trace = go.Ohlc(x=df['Date'],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'],
                    increasing=dict(line=dict(color='#58FA58')),
                    decreasing=dict(line=dict(color='#FA5858')))

    layout = {
        'title': f'{stock_name} Historical Price',
        'xaxis': {'title': 'Date',
                  'rangeslider': {'visible': False}},
        'yaxis': {'title': f'Price'}
    }

    data = [trace]

    fig = go.Figure(data=data, layout=layout)
    fig.show()


plot_ohlc(df_org)
# endregion

In [23]:
# region Sample data

df_org.head(10)

# endregion

,Date,Close,Open,High,Low,Adj Close,Volume,Change
0,2013-05-02,2275.669922,2272.100098,2281.429932,2262.080078,2275.655273,0,0.000000
1,2013-05-03,2308.330078,2280.840088,2326.570068,2280.840088,2308.315186,0,0.014352
2,2013-05-06,2335.169922,2315.649902,2340.340088,2315.649902,2335.154785,0,0.011627
3,2013-05-07,2339.800049,2331.699951,2344.750000,2322.889893,2339.784912,0,0.001983
4,2013-05-08,2351.070068,2347.000000,2360.429932,2339.639893,2351.054932,0,0.004817
5,2013-05-09,2337.070068,2351.790039,2352.719971,2324.979980,2337.054932,0,-0.005955
6,2013-05-10,2351.610107,2332.719971,2352.520020,2329.580078,2351.594971,0,0.006221
7,2013-05-13,2346.489990,2352.510010,2355.840088,2335.429932,2346.474854,0,-0.002177
8,2013-05-14,2320.360107,2343.320068,2344.449951,2308.729980,2320.345215,0,-0.011136
9,2013-05-15,2328.510010,2320.439941,2328.909912,2316.760010,2328.495117,0,0.003512


In [0]:
# region Const
# Declare const
input_col = ['Close', 'Open', 'High', 'Low', 'Adj Close', 'Change']
output_col = ['Close']
# input_col = ['Close', 'Open', 'High', 'Low', 'Adj Close', 'Volume']
# output_col = ['Close']

time_col = ['Date']

# Input dimension
input_dim = len(input_col)
# Output dimension
output_dim = len(output_col)

# Number of session to prediction as one time
prediction_size = 1
# For each time model is train, the first is display
sample_display_test_size = 5
# windows size
windows_size = 7


# endregion

In [0]:
# region Declare model
# declare model
def softMaxAxis1(x):
    return softmax(x, axis=1)


def get_model(input_dim, window_size, output_dim, lstm_layer_count=5, drop_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=128, input_shape=(window_size, input_dim), return_sequences=True))
    model.add(Dropout(rate=drop_rate))

    for i in range(lstm_layer_count - 2):
        model.add(LSTM(units=128, return_sequences=True))
        model.add(Dropout(rate=drop_rate))
    
    model.add(LSTM(units=128, return_sequences=False))
    model.add(Dropout(rate=drop_rate))
    model.add(Dense(output_dim, activation='linear'))
    opt = optimizers.Adam(lr=0.05, beta_1=0.99, beta_2=0.999)
    model.compile(loss='MSE', optimizer='adam')
    
    return model


# endregion

In [0]:
# region Error metric
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_square_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    return np.mean((y_true - y_pred) / y_true)


def relative_root_mean_square_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    res = (y_true - y_pred) / y_true
    res = np.power(res, 2)
    res = np.mean(res)
    res = math.sqrt(res)

    return res


# endregion

In [0]:
# region Data preprocessing
# reprocessing data
def next_window(df, i, windows_size, prediction_size, input_col, output_col, time_col):
    '''Generates the next data window from the given index location i'''
    window = df[i: i + windows_size + prediction_size]
    x = window[input_col][:-prediction_size]
    y = window[output_col][-prediction_size:]
    y_time = window[time_col][-prediction_size:]
    return x, y, y_time

def smooting_data(df, window_size):
    return df.ewm(span=window_size).mean()

def preprocessing_data(df, windows_size, prediction_size, input_col, output_col, time_col):
    '''
    Create x, y train data windows
    Warning: batch method, not generative, make sure you have enough memory to
    load data, otherwise use generate_training_window() method.
    '''


    data_x = []
    data_y = []
    data_y_time = []
    for i in range(len(df) - windows_size - prediction_size):
        x, y, y_time = next_window(df, i, windows_size, prediction_size, input_col, output_col, time_col)
        data_x.append(x.values)
        data_y.append(y.values)
        data_y_time.append(y_time)

    time = pd.concat(data_y_time)

    return np.array(data_x), np.array(data_y), time.values


def split_train_test_data(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=False)

    return X_train, y_train, X_valid, y_valid


# endregion

In [0]:
# region Model train
# Trainning model
def train_model(model, X_train, y_train, stock_name):
    if not IN_COLAB:
        model_save_fname = os.path.join(model_dir, '%s.h5' % (stock_name))
    else:
        model_save_fname = os.path.join(model_dir, '%s-%s.h5' % (stock_name, datetime.now().strftime('%d%m%Y_%H%M%S')))
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=100),
        ModelCheckpoint(filepath=model_save_fname, monitor='val_loss', save_best_only=True)
    ]
    history = model.fit(
        X_train,
        y_train,
        epochs=1000,
        batch_size=16,
        validation_split=1.0/7,
        verbose=1,
        callbacks=callbacks,
        shuffle=False)
    model.save(model_save_fname)
    
    return history


# endregion

In [0]:
def plot_test_result(test_result, stock_name):
    # Plotly
    trace0 = go.Scatter(
        x=test_result.index,
        y=test_result['Close'],
        name='Thực tế',
        line=dict(
            color=('#5042f4'),
            width=2)
    )

    trace1 = go.Scatter(
        x=test_result.index,
        y=test_result['Prediction'],
        name='Dự đoán',
        line=dict(
            color=('#005b4e'),
            width=2,
            dash='dot'
        )  # dash options include 'dash', 'dot', and 'dashdot'
    )

    data = [trace0, trace1]

    # Edit the layout
    layout = dict(title='Biểu đồ dự đoán',
                  xaxis=dict(title='Date'),
                  yaxis=dict(title='Price'),
                  paper_bgcolor='#FFF9F5',
                  plot_bgcolor='#FFF9F5'
                  )

    fig = go.Figure(data=data, layout=layout)
    fig.show()
        
# endregion

In [21]:
df = df_org.copy()
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=False)
# train_time, test_time = train_test_split(time, test_size=0.2, shuffle=False)

model = get_model(input_dim, windows_size, output_dim)
 

start = timer()

# Handle data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_cols = scaler.fit(df_train[input_col])
scaled_cols = scaler.transform(df_train[input_col])
df_train[input_col] = scaled_cols

X_train, y_train, time_train = preprocessing_data(df_train, windows_size, prediction_size, input_col, output_col, time_col)


# Reshape data
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1]))

# Perform n_train
history = train_model(model, X_train, y_train, stock_name)

run_time = timer() - start


Train on 569 samples, validate on 95 samples
Epoch 1/1000
569/569 [==============================] - 10s 17ms/sample - loss: 0.0193 - val_loss: 0.0113
Epoch 2/1000
569/569 [==============================] - 3s 5ms/sample - loss: 0.0744 - val_loss: 0.0900
Epoch 3/1000
569/569 [==============================] - 3s 5ms/sample - loss: 0.1026 - val_loss: 0.0357
Epoch 4/1000
569/569 [==============================] - 3s 6ms/sample - loss: 0.0100 - val_loss: 0.0201
Epoch 5/1000
569/569 [==============================] - 3s 5ms/sample - loss: 0.0241 - val_loss: 0.0190
Epoch 6/1000
569/569 [==============================] - 4s 6ms/sample - loss: 0.0508 - val_loss: 0.0080
Epoch 7/1000
569/569 [==============================] - 3s 5ms/sample - loss: 0.0320 - val_loss: 0.0097
Epoch 8/1000
569/569 [==============================] - 3s 6ms/sample - loss: 0.0091 - val_loss: 0.0097
Epoch 9/1000
569/569 [==============================] - 3s 5ms/sample - loss: 0.0188 - val_loss: 0.0273
Epoch 10/1000
569

In [26]:
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=False)
scaled_cols = scaler.transform(df_test[input_col])
df_test[input_col] = scaled_cols

X_test, y_test, time_test = preprocessing_data(df_test, windows_size, prediction_size, input_col, output_col, time_col)
# Reshape data
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1]))

# Test generated loss
y_pred = model.predict(X_test)
y_pred = np.repeat(y_pred, input_dim, axis=1)
y_pred = scaler.inverse_transform(y_pred)[:, [0]]
y_pred = pd.Series(y_pred.flatten())

test_result = pd.DataFrame(time_test, columns=['Date'])
test_result['Prediction'] = y_pred
test_result.set_index('Date', inplace=True)

test_result = test_result.join(df_org.set_index('Date'))
plot_test_result(test_result, stock_name)

score = model.evaluate(X_test, y_test, 10000, 1)

mae = mean_absolute_error(test_result['Close'], test_result['Prediction'])
mse = mean_squared_error(test_result['Close'], test_result['Prediction'])
mape = mean_absolute_percentage_error(test_result['Close'], test_result['Prediction'])
rrmse = relative_root_mean_square_error(test_result['Close'], test_result['Prediction'])

print(f'{stock_name} prediction for {prediction_size} day ahead')
print(f'MAE = {mae}')
print(f'MSE = {mse}')
print(f'MAPE = {mape}')
print(f'RRMSE = {rrmse}')
#loss = mape
loss = score
# write row

160/160 [==============================] - 0s 237us/sample - loss: 0.0016
000002.SS prediction for 1 day ahead
MAE = 124.84711608886718
MSE = 17758.771605227143
MAPE = 4.010582281386442
RRMSE = 0.042822966544232094


In [0]:
# Make future frame For 6 year, 3 year, 1 year, 1 month.

# Hyperparameter Tuning
#   + Train / test split valdiation
#   + Droprate
#   + Activation
#   + Number of layer

# Agents 
# Stock List